# Landsat Collection 2 downloader
michael.brechbuehler@eawag.ch

This notebook uses the [landsatxplore](https://github.com/yannforget/landsatxplore) python package to download Landsat Collection 2 scenes directly from USGS.

### Setup environment
To setup a new environment (using conda):
> conda create --name landsatxplore -c conda-forge python=3 notebook nb_conda_kernels geopandas hvplot cartopy geoviews

To setup a new environment (using conda and environment.yml):
> conda env create -n landsatxplore --file environment.yml

### Imports

In [168]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import shapely
import hvplot.pandas
from tqdm import tqdm
from pathlib import Path
import tarfile
from contextlib import closing

# import landsatxplore
import landsatxplore
from landsatxplore.earthexplorer import EarthExplorer
from landsatxplore.api import API

## 1. Search query

### USGS EROS credentials
To allow queries and file downloads from the USGS EROS webserver a free account is necessary (https://ers.cr.usgs.gov/register)

In [2]:
# Your USGS  credentials
username = "Mibrechb"
password = "1LRzp3#MRb2iUlO3"

Now we can set the desired search parameters for our Landsat query. We also have to define which Landsat collection to use.

| Dataset Name | Dataset ID |
| :-- | -- |
| Landsat 5 TM Collection 2 Level 1 | `landsat_tm_c2_l1` |
| Landsat 5 TM Collection 2 Level 2 | `landsat_tm_c2_l2` |
| Landsat 7 ETM+ Collection 2 Level 1 | `landsat_etm_c2_l1` |
| Landsat 7 ETM+ Collection 2 Level 2 | `landsat_etm_c2_l2` |
| Landsat 8 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 8 Collection 2 Level 2 | `landsat_ot_c2_l2` |
| Landsat 9 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 9 Collection 2 Level 2 | `landsat_ot_c2_l2` |

### Search parameters

In [124]:
# Search parameters
bbox = (6.117096, 46.189198, 6.938324 ,46.544463) # (lonmin, latmin, lonmax, latmax)

search_params = {
    'dataset': 'landsat_ot_c2_l2',
    'bbox': bbox, 
    'start_date': '2023-01-01',
    'end_date': '2023-12-31',
    'months': [1,2,3,4,5,6,7,8,9,10,11,12],
    'max_cloud_cover': 100,
    'max_results': 1000, # Defaults to 100 if not specified
}

### Plot ROI

In [125]:
# Plot bounding box on map
bbox_poly = shapely.geometry.box(*bbox, ccw=True)
df_bbox = gpd.GeoDataFrame([], geometry=[bbox_poly])

plot_bbox = df_bbox.hvplot(title='ROI bounding box', geo=True, 
                           fill_alpha=0.2, color='red', line_color='red', 
                           #tiles='CartoLight'
                           tiles='OSM', label='ROI'
                          )

plot_bbox

:Overlay
   .WMTS.I       :WMTS   [Longitude,Latitude]
   .Polygons.ROI :Polygons   [Longitude,Latitude]

### Start search query

In [126]:
# initialize a new API instance
api = API(username, password)

# search for Landsat 8-9 L2C2 scenes
scenes = api.search(**search_params)

# logout
api.logout()

# create a GeoDataFrame from the returned metadata
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date', 'spatial_coverage']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes['satellite'] = df_scenes.satellite.apply(lambda x: f'Landsat-{x}')
df_scenes['tile'] = df_scenes.display_id.apply(lambda x: x.split('_')[2][:3]+'/'+x.split('_')[2][3:])
gdf_scenes = gpd.GeoDataFrame(df_scenes.drop(columns='spatial_coverage'), geometry=df_scenes.spatial_coverage, crs='EPSG:4326')

gdf_scenes

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date,tile,geometry
0,LC09_L2SP_195028_20231031_20231101_02_T1,195,28,Landsat-9,79,2023-10-31,195/028,"POLYGON ((5.92512 45.38061, 8.28257 44.95382, ..."
1,LC08_L2SP_195028_20231023_20231031_02_T1,195,28,Landsat-8,100,2023-10-23,195/028,"POLYGON ((5.92204 45.38179, 8.27922 44.95240, ..."
2,LC09_L2SP_196027_20231022_20231024_02_T1,196,27,Landsat-9,20,2023-10-22,196/027,"POLYGON ((4.84587 46.80426, 7.26345 46.37038, ..."
3,LC09_L2SP_196028_20231022_20231024_02_T1,196,28,Landsat-9,23,2023-10-22,196/028,"POLYGON ((4.35110 45.38080, 6.70887 44.95390, ..."
4,LC09_L2SP_195028_20231015_20231016_02_T1,195,28,Landsat-9,38,2023-10-15,195/028,"POLYGON ((5.89642 45.38065, 8.25415 44.95371, ..."
...,...,...,...,...,...,...,...,...
107,LC08_L2SP_196027_20230115_20230131_02_T2,196,27,Landsat-8,96,2023-01-15,196/027,"POLYGON ((4.87241 46.80517, 7.28974 46.36939, ..."
108,LC08_L2SP_196028_20230115_20230131_02_T2,196,28,Landsat-8,100,2023-01-15,196/028,"POLYGON ((4.37817 45.38130, 6.73570 44.95248, ..."
109,LC08_L2SP_195028_20230108_20230124_02_T2,195,28,Landsat-8,100,2023-01-08,195/028,"POLYGON ((5.91210 45.38145, 8.26950 44.95259, ..."
110,LC09_L2SP_196027_20230107_20230314_02_T1,196,27,Landsat-9,11,2023-01-07,196/027,"POLYGON ((4.84817 46.80420, 7.26584 46.37081, ..."


### Plot available tiles over ROI

In [127]:
gdf_tiles = gdf_scenes.groupby('tile').first().reset_index()[['tile', 'wrs_path', 'wrs_row', 'geometry']].set_crs('EPSG:4326')
gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)

plot_labels = gdf_labels.to_crs("EPSG:3857").assign(x=lambda df: df.geometry.x, y=lambda df: df.geometry.y).hvplot.labels(text="tile", x="x", y="y", text_color='white')

plot_tiles = gdf_tiles.hvplot(title='Available tiles over ROI', geo=True,
                              fill_alpha=0.3, line_color='blue',
                              height=800, tiles='OSM', label='Landsat tile'
                             ) * plot_bbox * plot_labels

plot_tiles

C:\Users\Michael\AppData\Local\Temp\ipykernel_23392\3331222002.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)


:Overlay
   .WMTS.I                :WMTS   [Longitude,Latitude]
   .Polygons.Landsat_tile :Polygons   [Longitude,Latitude]
   .WMTS.II               :WMTS   [Longitude,Latitude]
   .Polygons.ROI          :Polygons   [Longitude,Latitude]
   .Labels.I              :Labels   [x,y]   (tile)

### Plot all available scenes

In [156]:
plot_scenes = df_scenes.hvplot.line(title=f'Available Landsat C2 scenes over ROI\n{search_params["start_date"]} to {search_params["end_date"]}, n={df_scenes.shape[0]}',
    x='acquisition_date', y='cloud_cover', color='grey', grid=True) *\
                df_scenes.hvplot.scatter(x='acquisition_date', y='cloud_cover', c='satellite',
                                       hover_cols=['tile', 'acquisition_date'],
                                       xlabel='Acq. date', ylabel='Cloud cover (%)', marker='o')
plot_scenes

:Overlay
   .Curve.I   :Curve   [acquisition_date]   (cloud_cover)
   .Scatter.I :Scatter   [acquisition_date]   (cloud_cover,satellite,tile)

## 2. Download scenes
### Filter and select scene IDs

In [161]:
# Select scene ids to download from dataframe
#ids = df_scenes.display_id.values # all
df_scenes_filt = df_scenes.loc[(df_scenes.cloud_cover < 10) & (df_scenes.tile=='196/028')] # filter by cloudcover and tile
ids = df_scenes_filt.display_id.values
df_scenes_filt

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date,spatial_coverage,tile
9,LC09_L2SP_196028_20231006_20231007_02_T1,196,28,Landsat-9,2,2023-10-06,"POLYGON ((4.37026 45.38081, 6.72823 44.95378, ...",196/028
21,LC09_L2SP_196028_20230904_20230906_02_T1,196,28,Landsat-9,8,2023-09-04,"POLYGON ((4.36819 45.38073, 6.72523 44.95371, ...",196/028
27,LC09_L2SP_196028_20230819_20230821_02_T1,196,28,Landsat-9,2,2023-08-19,"POLYGON ((4.38865 45.3808, 6.74557 44.95369, 7...",196/028
30,LC08_L2SP_196028_20230811_20230818_02_T1,196,28,Landsat-8,1,2023-08-11,"POLYGON ((4.39496 45.38193, 6.75219 44.95209, ...",196/028
48,LC08_L2SP_196028_20230624_20230630_02_T1,196,28,Landsat-8,1,2023-06-24,"POLYGON ((4.39833 45.382, 6.7553 44.95215, 7.4...",196/028
78,LC08_L2SP_196028_20230405_20230412_02_T1,196,28,Landsat-8,4,2023-04-05,"POLYGON ((4.3582 45.38097, 6.71535 44.95283, 7...",196/028
96,LC08_L2SP_196028_20230216_20230223_02_T1,196,28,Landsat-8,2,2023-02-16,"POLYGON ((4.34142 45.38107, 6.69906 44.95245, ...",196/028
99,LC09_L2SP_196028_20230208_20230310_02_T1,196,28,Landsat-9,9,2023-02-08,"POLYGON ((4.30799 45.38013, 6.66563 44.95397, ...",196/028
102,LC08_L2SP_196028_20230131_20230208_02_T1,196,28,Landsat-8,4,2023-01-31,"POLYGON ((4.33643 45.3814, 6.69443 44.95256, 7...",196/028


### Download .tar files

In [209]:
# set output directory
path_output_dir = Path(r'E:/landsat_c2l2')

# initialize the API
ee = EarthExplorer(username, password)

# download the scenes
for id in tqdm(ids, total=len(ids), desc="Total download progress", position=-1):
    path_output_file = path_output_dir.joinpath(id+'.tar')
    try:
        ee.download(id, output_dir=path_output_dir)
        print('{} successful'.format(id))

  # aditional error handling
    except:
        if path_output_file.exists():
            print(f'{id} error but file exists')
        else:
            print(f'{id} error')

ee.logout()


Total progress::   0%|                                                                           | 0/9 [00:00<?, ?it/s]

Download failed with dataset id 1 of 3. Re-trying with the next one.
Download failed with dataset id 2 of 3. Re-trying with the next one.


100%|█████████████████████████████████████████████████████████████████████████████| 0.98G/0.98G [01:37<00:00, 10.9MB/s]

Total progress::  11%|███████▎                                                          | 1/9 [01:44<13:55, 104.41s/it]

LC09_L2SP_196028_20231006_20231007_02_T1 successful
Download failed with dataset id 1 of 3. Re-trying with the next one.
Download failed with dataset id 2 of 3. Re-trying with the next one.


100%|█████████████████████████████████████████████████████████████████████████████| 1.02G/1.02G [00:38<00:00, 28.7MB/s]

Total progress::  22%|██████████████▉                                                    | 2/9 [02:42<09:27, 81.08s/it]


LC09_L2SP_196028_20230904_20230906_02_T1 successful


### Extract .tar files

In [210]:
# Extract .tar files into folders
paths_output = list(path_output_dir.glob('*.tar'))

for path in tqdm(paths_output, desc="Total extraction progress"):
    with closing(tarfile.open(path)) as fl:
        path_output_folder = path_output_dir.joinpath(path.name.split('.')[0])
        path_output_folder.mkdir(parents=True, exist_ok=True)
        fl.extractall(path_output_folder)
    path.unlink()

Total extraction progress: 100%|█████████████████████████████████████████████████████████| 2/2 [00:20<00:00, 10.08s/it]
